In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pt
import seaborn as sns
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import pickle

In [15]:
!pip install pandas --upgrade

                                              0.0/11.6 MB ? eta -:--:--
                                             0.0/11.6 MB 330.3 kB/s eta 0:00:36
                                             0.1/11.6 MB 787.7 kB/s eta 0:00:15
     --                                       0.7/11.6 MB 5.3 MB/s eta 0:00:03
     ------                                   1.8/11.6 MB 10.7 MB/s eta 0:00:01
     -----------                              3.2/11.6 MB 14.8 MB/s eta 0:00:01
     ---------------                          4.4/11.6 MB 16.5 MB/s eta 0:00:01
     ------------------                       5.5/11.6 MB 17.6 MB/s eta 0:00:01
     ----------------------                   6.6/11.6 MB 18.5 MB/s eta 0:00:01
     ---------------------------              7.9/11.6 MB 19.5 MB/s eta 0:00:01
     -----------------------------            8.7/11.6 MB 19.2 MB/s eta 0:00:01
     --------------------------------         9.5/11.6 MB 19.0 MB/s eta 0:00:01
     -----------------------------------     10.5

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\gokul\\anaconda3\\envs\\gokul1\\Lib\\site-packages\\~andas\\_libs\\algos.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
books = pd.read_csv(r'data\BX-Books.csv', encoding='latin-1')

In [3]:
rating = pd.read_csv(r'data\BX-Book-Ratings2.csv')

In [4]:
users = pd.read_csv(r'data\Users1.csv')

In [5]:
books.head(5)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999.0,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [6]:
rating.head(5)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [7]:
users.head(5)

,User-ID,Age
0,1.0,NaN
1,2.0,18.0
2,3.0,NaN
3,4.0,17.0
4,5.0,NaN


In [12]:
print(books.shape)
print(rating.shape)
print(users.shape)

(271379, 8)
(1048575, 3)
(278859, 2)


In [8]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [9]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-L']]

In [10]:
books.head(5)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991.0,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999.0,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999.0,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [11]:
books.rename(columns={"Book-Title":"title","Book-Author":"author","Year-Of-Publication":"year",
                     "Publisher":"publisher","Image-URL-L":"img_url"}, inplace=True)

In [12]:
books['year'] = books['year'].fillna(0)
books['year']=books['year'].astype(int)

In [13]:
books.head(3)

,ISBN,title,author,year,publisher,img_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...


In [14]:
users['User-ID']=users['User-ID'].fillna(0)
users['User-ID']=users['User-ID'].astype(int)

In [15]:
users

,User-ID,Age
0,1,NaN
1,2,18.0
2,3,NaN
3,4,17.0
4,5,NaN
...,...,...
278854,278854,NaN
278855,278855,NaN
278856,278856,NaN
278857,278857,NaN


In [16]:
rating['Book-Rating']=rating['Book-Rating'].fillna(0)
rating['Book-Rating']=rating['Book-Rating'].astype(int)

In [17]:
rating

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1048570,250764,0451410777,0
1048571,250764,0452264464,8
1048572,250764,048623715X,0
1048573,250764,0486256588,0


In [18]:
rating.rename(columns={"User-ID":"user_id","Book-Rating":"rating"},inplace=True)

In [19]:
rating.head(3)

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0


In [20]:
rating['user_id'].value_counts()

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
104999        1
105002        1
105008        1
105014        1
123969        1
Name: count, Length: 95513, dtype: int64

In [21]:
rating['user_id'].unique().shape

(95513,)

In [22]:
a = rating['user_id'].value_counts() > 200
a

user_id
11676      True
198711     True
153662     True
98391      True
35859      True
          ...  
104999    False
105002    False
105008    False
105014    False
123969    False
Name: count, Length: 95513, dtype: bool

In [23]:
a[a].shape

(815,)

In [24]:
b=a[a].index
b

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
        88793,  33145, 116122,   9856,  73681,  28634,  59727, 188951, 155916,
        44296],
      dtype='int64', name='user_id', length=815)

In [25]:
ratings= rating[rating['user_id'].isin(b)]

In [26]:
ratings.head(5)

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [27]:
ratings.shape

(482728, 3)

In [28]:
ratings_with_books=ratings.merge(books, on="ISBN")

In [29]:
ratings_with_books.head(4)

,user_id,ISBN,rating,title,author,year,publisher,img_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...
3,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...


In [30]:
ratings_with_books.shape

(446772, 8)

In [31]:
num_rating =ratings_with_books.groupby('title')['rating'].count().reset_index()

In [32]:
num_rating.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [33]:
num_rating.rename(columns={"rating":"num_of_rating"},inplace=True)

In [34]:
num_rating.head()

,title,num_of_rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [35]:
final_rating=ratings_with_books.merge(num_rating, on='title')

In [42]:
final_rating.head(4)

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,74
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...,7
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...,1
3,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...,1


In [36]:
final_rating.shape

(446772, 9)

In [37]:
final_rating=final_rating[final_rating['num_of_rating']>=50]

In [38]:
final_rating.sample(5)

,user_id,ISBN,rating,title,author,year,publisher,img_url,num_of_rating
382324,223154,0316781266,0,The Last Time They Met : A Novel,Anita Shreve,2002,Back Bay Books,http://images.amazon.com/images/P/0316781266.0...,83
305347,177458,0061009059,8,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,1995,HarperTorch,http://images.amazon.com/images/P/0061009059.0...,99
338479,197775,0140298479,9,Bridget Jones: The Edge of Reason,Helen Fielding,2001,Penguin Books,http://images.amazon.com/images/P/0140298479.0...,66
235406,135149,0446606189,0,Cat &amp; Mouse (Alex Cross Novels),James Patterson,1998,Warner Books,http://images.amazon.com/images/P/0446606189.0...,65
117264,68555,0385502532,0,Drowning Ruth,Christina Schwarz,2000,Doubleday,http://images.amazon.com/images/P/0385502532.0...,56


In [39]:
final_rating.to_csv('C:\\Users\\gokul\\Desktop\\Constient_Global_Solution\\Book_recommendation\\data\\final_rating.csv', index=False)

In [42]:
final_rating.shape

(49066, 9)

In [41]:
final_rating.drop_duplicates(['user_id','title'],inplace=True)

In [43]:
final_rating.shape

(49066, 9)

In [44]:
 book_pivot=final_rating.pivot_table(columns='user_id', index='title', values='rating')

In [45]:
book_pivot.to_csv('C:\\Users\\gokul\\Desktop\\Constient_Global_Solution\\Book_recommendation\\data\\book_pivot.csv', index=False)

In [50]:
book_pivot.shape

(626, 804)

In [51]:
book_pivot.fillna(0, inplace=True)

In [54]:
book_pivot.head(3)

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,249628,249862,249894,250184,250405,250764,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
book_sparde = csr_matrix(book_pivot)

In [56]:
book_sparde 

<626x804 sparse matrix of type '<class 'numpy.float64'>'
	with 12391 stored elements in Compressed Sparse Row format>

In [57]:
model= NearestNeighbors(algorithm='ball_tree')

In [58]:
model.fit(book_sparde)

C:\Users\gokul\anaconda3\envs\gokul1\Lib\site-packages\sklearn\neighbors\_base.py:564: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")


NearestNeighbors(algorithm='ball_tree')

In [59]:
distance,suggestion=model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1), n_neighbors=10)

In [60]:
distance

array([[ 0.        , 21.44761059, 24.31049156, 24.8394847 , 25.03996805,
        25.11971337, 25.23885893, 25.74878638, 26.26785107, 26.51414717]])

In [61]:
 suggestion

array([[237, 448, 577,  80, 331,   8, 108, 504, 162, 588]], dtype=int64)

In [62]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])

Index(['Invasion', 'The Cradle Will Fall',
       'Tom Clancy's Op-Center (Tom Clancy's Op Center (Paperback))',
       'CAT'S EYE', 'Pleading Guilty', 'A Civil Action', 'Cry Wolf',
       'The Most Wanted', 'Fat Tuesday', 'Vanished'],
      dtype='object', name='title')


In [63]:
book_pivot.index[9]

'A Day Late and a Dollar Short'

In [64]:
book_pivot.index

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Day Late and a Dollar Short',
       ...
       'Winter Moon', 'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wuthering Heights', 'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=626)

In [65]:
books_name =book_pivot.index

In [66]:
pickle.dump(model,open('data/model.pkl','wb'))
pickle.dump(books_name,open('data/books_name.pkl','wb'))
pickle.dump(final_rating,open('data/final_rating.pkl','wb'))
pickle.dump(book_pivot,open('data/book_pivot.pkl','wb'))

In [67]:
def recommend_book(books_name):
    book_id =np.where(book_pivot.index==books_name)[0][0]
    distance,suggestion=model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1), n_neighbors=10)
    
    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        for j in books:
            print(j)

In [74]:
books_name ="Pleading Guilty"
recommend_book(books_name)

Pleading Guilty
CAT'S EYE
Journey
The Most Wanted
4 Blondes
The Cardinal of the Kremlin (Jack Ryan Novels)
Primary Colors: A Novel of Politics
Absolute Power
A Civil Action
Sleepers
